Temporal distribution of S2 & S1

In [1]:
STAC_API = "https://planetarycomputer.microsoft.com/api/stac/v1"

In [ ]:
S2_BANDS = ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12", "SCL"]

In [3]:
!stac-client search {STAC_API} \
    --collection sentinel-2-l2a \
    --bbox -80.09 8.95 -79.08 9.94 \
    --datetime 2022-01-01/2022-12-31 | stacterm cal

zsh:1: command not found: stacterm
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe


In [4]:
!stac-client search {STAC_API} \
    --collection sentinel-1-rtc \
    --bbox -80.09 8.95 -79.08 9.94 \
    --datetime 2022-01-01/2022-12-31 | stacterm cal

zsh:1: command not found: stacterm
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe


In [5]:
import random
import json
import os
from datetime import datetime, timedelta
from getpass import getpass

import geopandas as gpd
import leafmap
import numpy as np
import pystac
import pystac_client
import planetary_computer as pc
import shapely
from shapely.geometry import box, shape
import stackstac
import xarray as xr

In [47]:
california_tile = gpd.read_file("../../ca.geojson") #../data/raw/mgrs.fgb")

In [10]:
def random_date(start_year, end_year):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    days_between = (end_date - start_date).days
    random_days = random.randint(0, days_between)
    random_date = start_date + timedelta(days=random_days)
    return random_date

def get_week(year, month, day):
    date = datetime(year, month, day)
    start_of_week = date - timedelta(days=date.weekday())
    end_of_week = start_of_week + timedelta(days=6)
    start_date_str = start_of_week.strftime('%Y-%m-%d')
    end_date_str = end_of_week.strftime('%Y-%m-%d')
    return f"{start_date_str}/{end_date_str}"

In [11]:
date = random_date(2017, 2023)
YEAR = date.year 
MONTH = date.month
DAY = date.day
CLOUD = 50

sample = california_tile.sample(1)
CENTROID = sample.iloc[0].geometry.centroid
BBOX = sample.iloc[0].geometry.bounds

In [12]:
YEAR, MONTH, DAY, CENTROID, BBOX

(2017,
 10,
 1,
 (-125.3695358714707,
  32.232410022824496,
  -112.19041038128745,
  42.146036827423075))

In [13]:
week = get_week(YEAR, MONTH, DAY)
week

'2017-09-25/2017-10-01'

Query for S2 tile for the sample CENTROID in this week

In [14]:
catalog = pystac_client.Client.open(STAC_API, modifier=pc.sign_inplace)

search = catalog.search(
    collections=["sentinel-2-l2a"], 
    datetime=week,
    intersects={
        "type": "Point",
        "coordinates": [CENTROID.x, CENTROID.y]
    },
    sortby="eo:cloud_cover"
)

In [15]:
s2_items = search.get_all_items()
print(f"Found {len(s2_items)} items")

Found 1 items


In [50]:
s2_items_gdf = gpd.GeoDataFrame.from_features(s2_items.to_dict())

In [67]:
s2_gdf = items_gdf[s2_items_gdf.area == s2_items_gdf.area.max()] # Filter for the tile with maximum coverage

In [69]:
def plot_gdf(m, gdf, name, color):
    m.add_gdf(gdf, 
              layer_name=name,
              style={
                  "color": color,
                  "fillColor": color,
                  "fillOpacity": 0.2
              })

In [22]:
m = leafmap.Map(center=[0, 0], zoom=2)

In [23]:
plot_gdf(m, s2_gdf.iloc[[0]], "S2", "orange")

In [24]:
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

Query for S1 tile covering the S2 BBOX

In [26]:
BBOX = s2_gdf.iloc[0].geometry.bounds
search = catalog.search(
    collections=["sentinel-1-rtc"], 
    bbox=BBOX, 
    datetime=week
)
s1_items = search.item_collection()
print(f"Found {len(s1_items)} items")

Found 3 items


In [27]:
s1_gdf = gpd.GeoDataFrame.from_features(s1_items.to_dict())
s1_gdf

,geometry,datetime,platform,s1:shape,proj:bbox,proj:epsg,proj:shape,end_datetime,constellation,s1:resolution,...,sar:center_frequency,sar:resolution_range,s1:product_timeliness,sar:resolution_azimuth,sar:pixel_spacing_range,sar:observation_direction,sar:pixel_spacing_azimuth,sar:looks_equivalent_number,s1:instrument_configuration_ID,sat:platform_international_designator
0,"POLYGON ((-119.77434 37.97362, -119.44115 36.4...",2017-09-30T01:50:29.137225Z,SENTINEL-1B,"[28269, 20949]","[256310.0, 4038310.0, 539000.0, 4247800.0]",32611,"[20949, 28269]",2017-09-30 01:50:41.637079+00:00,Sentinel-1,high,...,5.405,20,Fast-24h,22,10,right,10,4.4,1,2016-025A
1,"POLYGON ((-118.14705 35.27623, -120.90462 35.6...",2017-09-29T14:00:04.827877Z,SENTINEL-1A,"[28637, 21681]","[144770.0, 3904280.0, 431140.0, 4121090.0]",32611,"[21681, 28637]",2017-09-29 14:00:17.327081+00:00,Sentinel-1,high,...,5.405,20,Fast-24h,22,10,right,10,4.4,5,2014-016A
2,"POLYGON ((-117.72869 37.03082, -117.44051 38.2...",2017-09-29T13:59:39.827976Z,SENTINEL-1A,"[28471, 21659]","[177980.0, 4070510.0, 462690.0, 4287100.0]",32611,"[21659, 28471]",2017-09-29 13:59:52.327180+00:00,Sentinel-1,high,...,5.405,20,Fast-24h,22,10,right,10,4.4,5,2014-016A


In [28]:
# TODO: Find a way to get minimal number of S1 tiles to cover the entire S2 bbox
s1_gdf["overlap"] = s1_gdf.intersection(box(*BBOX)).area
s1_gdf.sort_values(by="overlap", inplace=True)

In [29]:
plot_gdf(m, s1_gdf, "S1", "green")

In [30]:
m

Map(bottom=6696.0, center=[35.7830062563083, -112.9929280796005], controls=(ZoomControl(options=['position', '…

Query for COP-DEMs covering the S2 BBOX

In [94]:
search = catalog.search(
    collections=["cop-dem-glo-30"], bbox=BBOX
)
dem_items = search.item_collection()
print(f"Found {len(dem_items)} items")

Found 4 items


In [95]:
dem_gdf = gpd.GeoDataFrame.from_features(dem_items.to_dict())

dem_gdf.set_crs(epsg=4326, inplace=True)
dem_gdf = dem_gdf.to_crs(epsg=s2_items[0].properties["proj:epsg"])

In [142]:
dem_gdf

,geometry,gsd,datetime,platform,proj:epsg,proj:shape,proj:transform
0,"POLYGON ((322025.774 4096757.729, 324384.765 4...",30,2021-04-22T00:00:00Z,TanDEM-X,4326,"[3600, 3600]","[0.0002777777777777778, 0.0, -120.000138888888..."
1,"POLYGON ((411010.479 4095355.229, 412189.553 4...",30,2021-04-22T00:00:00Z,TanDEM-X,4326,"[3600, 3600]","[0.0002777777777777778, 0.0, -119.000138888888..."
2,"POLYGON ((319721.138 3985813.873, 322025.774 4...",30,2021-04-22T00:00:00Z,TanDEM-X,4326,"[3600, 3600]","[0.0002777777777777778, 0.0, -120.000138888888..."
3,"POLYGON ((409858.590 3984426.322, 411010.479 4...",30,2021-04-22T00:00:00Z,TanDEM-X,4326,"[3600, 3600]","[0.0002777777777777778, 0.0, -119.000138888888..."


In [96]:
plot_gdf(m, dem_gdf, "DEM", "blue")

In [97]:
m

Map(bottom=6696.0, center=[35.7830062563083, -112.9929280796005], controls=(ZoomControl(options=['position', '…

In [144]:
da_sen1: xr.DataArray = stackstac.stack(
    items=s1_items[1:], # To only accept the same orbit state and date. Need better way to do this.
    assets=["vh", "vv"],  # SAR polarizations
    epsg=26910,  # UTM Zone 10N
    bounds_latlon=BBOX,  # W, S, E, N
    xy_coords="center",  # pixel centroid coords instead of topleft corner
    dtype=np.float32,
    fill_value=np.nan,
)

In [145]:
# %%
# To fix TypeError: Invalid value for attr 'spec'
da_sen1.attrs["spec"] = str(da_sen1.spec)

# To fix ValueError: unable to infer dtype on variable None
for key, val in da_sen1.coords.items():
    if val.dtype == "object":
        print("Deleting", key)
        da_sen1 = da_sen1.drop_vars(names=key)

# Create xarray.Dataset datacube with VH and VV channels from SAR
da_vh: xr.DataArray = da_sen1.sel(band="vh", drop=True).rename("vh")
da_vv: xr.DataArray = da_sen1.sel(band="vv", drop=True).rename("vv")
ds_sen1: xr.Dataset = xr.merge(objects=[da_vh, da_vv], join="override")

#print(ds_sen1)

Deleting sar:polarizations
Deleting raster:bands


In [ ]:
da_sen1 = da_sen1.drop([da_sen1.time[0].values], dim='time') # Remove first scene which has ascending orbit

In [147]:
da_sen1

<xarray.DataArray 'stackstac-f67c69b8cb89287d09f6c6d2ccfd3af7' (time: 1,
                                                                band: 2,
                                                                y: 11031,
                                                                x: 11032)>
dask.array<getitem, shape=(1, 2, 11031, 11032), dtype=float32, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/37)
  * time                                   (time) datetime64[ns] 2017-09-29T1...
    id                                     (time) <U66 'S1A_IW_GRDH_1SDV_2017...
  * band                                   (band) <U2 'vh' 'vv'
  * x                                      (x) float64 8.273e+05 ... 9.442e+05
  * y                                      (y) float64 4.211e+06 ... 4.094e+06
    s1:orbit_source                        <U6 'RESORB'
    ...                                     ...
    sar:frequency_band                     <U1 'C'
    s1:datatake_id                         <U6 '128371'
    sar:center_frequency                   float64 5.405
    title                                  (band) <U41 'VH: vertical transmit...
    description                            (band) <U173 'Terrain-corrected ga...
    epsg                                   int64 26910
Attributes:
    spec:           RasterSpec(epsg=26910, bounds=(827304.1726848192, 4094320...
    crs:            epsg:26910
    transform:      | 10.60, 0.00, 827304.17|\n| 0.00,-10.60, 4211253.32|\n| ...
    resolution_xy:  (10.60034816688858, 10.60034816712141)

In [148]:
da_sen2: xr.DataArray = stackstac.stack(
    items=s2_items[0],
    epsg=26910,  # UTM Zone 10N
    assets=S2_BANDS, 
    bounds_latlon=BBOX,  # W, S, E, N
    resolution=10,  # Spatial resolution of 10 metres
    xy_coords="center",  # pixel centroid coords instead of topleft corner
    dtype=np.float32,
    fill_value=np.nan,
)

In [149]:
da_sen2

<xarray.DataArray 'stackstac-d51c043dad8f15dc7472c7cca73ce3f2' (time: 1,
                                                                band: 11,
                                                                y: 11693,
                                                                x: 11694)>
dask.array<fetch_raster_window, shape=(1, 11, 11693, 11694), dtype=float32, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/44)
  * time                                     (time) datetime64[ns] 2017-09-29...
    id                                       (time) <U54 'S2B_MSIL2A_20170929...
  * band                                     (band) <U3 'B02' 'B03' ... 'SCL'
  * x                                        (x) float64 8.273e+05 ... 9.442e+05
  * y                                        (y) float64 4.211e+06 ... 4.094e+06
    s2:product_type                          <U7 'S2MSI2A'
    ...                                       ...
    proj:bbox                                object {300000.0, 4090200.0, 420...
    gsd                                      (band) float64 10.0 10.0 ... 20.0
    common_name                              (band) object 'blue' ... None
    center_wavelength                        (band) object 0.49 0.56 ... None
    full_width_half_max                      (band) object 0.098 0.045 ... None
    epsg                                     int64 26910
Attributes:
    spec:        RasterSpec(epsg=26910, bounds=(827300, 4094320, 944240, 4211...
    crs:         epsg:26910
    transform:   | 10.00, 0.00, 827300.00|\n| 0.00,-10.00, 4211250.00|\n| 0.0...
    resolution:  10

In [169]:
da_dem: xr.DataArray = stackstac.stack(
    items=dem_items,
    epsg=26910,  # UTM Zone 10N
    bounds_latlon=BBOX,  # W, S, E, N
    resolution=10,  # Spatial resolution of 10 metres
    xy_coords="center",  # pixel centroid coords instead of topleft corner
    dtype=np.float32,
    fill_value=np.nan,
)

In [158]:
da_dem

<xarray.DataArray 'stackstac-a7d63949c5206055c137da6326a00db1' (time: 4,
                                                                band: 1,
                                                                y: 11693,
                                                                x: 11694)>
dask.array<fetch_raster_window, shape=(4, 1, 11693, 11694), dtype=float32, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time        (time) datetime64[ns] 2021-04-22 2021-04-22 ... 2021-04-22
    id          (time) <U40 'Copernicus_DSM_COG_10_N37_00_W120_00_DEM' ... 'C...
  * band        (band) <U4 'data'
  * x           (x) float64 8.273e+05 8.273e+05 ... 9.442e+05 9.442e+05
  * y           (y) float64 4.211e+06 4.211e+06 ... 4.094e+06 4.094e+06
    platform    <U8 'TanDEM-X'
    proj:shape  object {3600}
    proj:epsg   int64 4326
    gsd         int64 30
    epsg        int64 26910
Attributes:
    spec:        RasterSpec(epsg=26910, bounds=(827300, 4094320, 944240, 4211...
    crs:         epsg:26910
    transform:   | 10.00, 0.00, 827300.00|\n| 0.00,-10.00, 4211250.00|\n| 0.0...
    resolution:  10

In [187]:
_, index = np.unique(da_dem['time'], return_index=True) # Remove redundant time
da_dem.isel(time=index)

<xarray.DataArray 'stackstac-8ae7a985a28ec34323dc613801ea8178' (time: 1,
                                                                band: 1,
                                                                y: 11693,
                                                                x: 11694)>
dask.array<getitem, shape=(1, 1, 11693, 11694), dtype=float32, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time        (time) datetime64[ns] 2021-04-22
    id          (time) <U40 'Copernicus_DSM_COG_10_N37_00_W120_00_DEM'
  * band        (band) <U4 'data'
  * x           (x) float64 8.273e+05 8.273e+05 ... 9.442e+05 9.442e+05
  * y           (y) float64 4.211e+06 4.211e+06 ... 4.094e+06 4.094e+06
    platform    <U8 'TanDEM-X'
    proj:shape  object {3600}
    proj:epsg   int64 4326
    gsd         int64 30
    epsg        int64 26910
Attributes:
    spec:        RasterSpec(epsg=26910, bounds=(827300, 4094320, 944240, 4211...
    crs:         epsg:26910
    transform:   | 10.00, 0.00, 827300.00|\n| 0.00,-10.00, 4211250.00|\n| 0.0...
    resolution:  10

In [189]:
da_dem.isel(time=index)

<xarray.DataArray 'stackstac-8ae7a985a28ec34323dc613801ea8178' (time: 1,
                                                                band: 1,
                                                                y: 11693,
                                                                x: 11694)>
dask.array<getitem, shape=(1, 1, 11693, 11694), dtype=float32, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time        (time) datetime64[ns] 2021-04-22
    id          (time) <U40 'Copernicus_DSM_COG_10_N37_00_W120_00_DEM'
  * band        (band) <U4 'data'
  * x           (x) float64 8.273e+05 8.273e+05 ... 9.442e+05 9.442e+05
  * y           (y) float64 4.211e+06 4.211e+06 ... 4.094e+06 4.094e+06
    platform    <U8 'TanDEM-X'
    proj:shape  object {3600}
    proj:epsg   int64 4326
    gsd         int64 30
    epsg        int64 26910
Attributes:
    spec:        RasterSpec(epsg=26910, bounds=(827300, 4094320, 944240, 4211...
    crs:         epsg:26910
    transform:   | 10.00, 0.00, 827300.00|\n| 0.00,-10.00, 4211250.00|\n| 0.0...
    resolution:  10

In [191]:
da_merge = xr.merge([da_sen2, da_sen1, da_dem.isel(time=index)], compat='override')

In [192]:
da_merge

<xarray.Dataset>
Dimensions:                                     (time: 3, band: 14, x: 22726,
                                                 y: 22724)
Coordinates: (12/70)
  * time                                        (time) datetime64[ns] 2017-09...
  * band                                        (band) <U4 'B02' 'B03' ... 'vv'
  * x                                           (x) float64 8.273e+05 ... 9.4...
  * y                                           (y) float64 4.094e+06 ... 4.2...
    id                                          (time) object nan ... nan
    s2:product_type                             <U7 'S2MSI2A'
    ...                                          ...
    s1:product_timeliness                       <U8 'Fast-24h'
    sar:frequency_band                          <U1 'C'
    s1:datatake_id                              <U6 '128371'
    sar:center_frequency                        float64 5.405
    description                                 (band) object nan ... 'Terrai...
    proj:shape                                  object {3600}
Data variables:
    stackstac-d51c043dad8f15dc7472c7cca73ce3f2  (time, band, y, x) float32 dask.array<chunksize=(3, 1, 834, 1990), meta=np.ndarray>
    stackstac-f67c69b8cb89287d09f6c6d2ccfd3af7  (time, band, y, x) float32 dask.array<chunksize=(3, 13, 1630, 2110), meta=np.ndarray>
    stackstac-8ae7a985a28ec34323dc613801ea8178  (time, band, y, x) float32 dask.array<chunksize=(3, 14, 834, 1990), meta=np.ndarray>
Attributes:
    spec:        RasterSpec(epsg=26910, bounds=(827300, 4094320, 944240, 4211...
    crs:         epsg:26910
    transform:   | 10.00, 0.00, 827300.00|\n| 0.00,-10.00, 4211250.00|\n| 0.0...
    resolution:  10